In [16]:
#Importamos librerias
import pandas as pd
import re

In [ ]:
#Importamos los datos de los restaurantes de google
google = pd.read_json('metadata-sitios/google_sitios.json')
google.head()

In [ ]:
#Examinamos un poco más
google.info()

Como primera medida vamos a normalizar los campos. Lo que buscamos es tener una tabla que contenga las siguientes columnas:

* Id_Restaurant
* Nombre
* Ciudad
* Estado
* Código postal
* Latitud
* Longitud
* Tipo (category)
* Atributos (MISC)

La información sobre la ciudad, estado y código postal se encuentra bajo el campo address del dataframe de google. Vamos a separarlo.

In [ ]:
#Quitamos los valores que no tengan el valor de address
restaurants_google = google.dropna(subset='address')
restaurants_google.reset_index(inplace=True)
restaurants_google.info()

In [ ]:
#Separamos y juntamos los datos
listado_fallas = []
localizaciones = []
for i in range(restaurants_google['address'].values.size):
    try:
        int(restaurants_google['address'].values[i][-1])
        if len(restaurants_google['address'].values[i].split(',')[-1].strip()) == 8:
            localizaciones.append([restaurants_google['gmap_id'].values[i],                                     #Id del restaurant
                                restaurants_google['address'].values[i].split(',')[-2].strip(),                 #Ciudad
                                restaurants_google['address'].values[i].split(',')[-1].strip().split()[0],      #Estado
                                restaurants_google['address'].values[i].split(',')[-1].strip().split()[1]])     #Código postal
        else:
            listado_fallas.append(i)                                    #Anoto cuantas direcciones no me dan los datos necesarios
    except:
        listado_fallas.append(i)
len(listado_fallas)

In [ ]:
#Armamos dataframe con estos datos
localizacion = pd.DataFrame(columns=['gmap_id','Ciudad','Estado','Cod_postal'],data=localizaciones)
localizacion.head()

In [ ]:
#Realizamos una comprobación. Vemos si algún valor falló
fallados = []
for i in range(localizacion.shape[0]):
    try:
        int(localizacion['Cod_postal'].iloc[i])                 #Si esto funciona quiere decir que no entraron caracteres extraños
    except:                                                     #bajo cod_postal
        print(i,': ',localizacion['Cod_postal'].iloc[i])
        fallados.append(localizacion['Cod_postal'].iloc[i])

In [ ]:
#No fallo. 12590 registros no cuentan con los valores que buscamos. Los observamos a ver si se puede hacer algo.
restaurants_google[restaurants_google.index.isin(listado_fallas)]

In [ ]:
#Observamos el campo address más de cerca
list(restaurants_google[restaurants_google.index.isin(listado_fallas)]['address'].values)

Vemos que hay muchos que tienen "United States" al final. Esto trae problemas en el algoritmo planteado. Modificando ligeramente podemos incorporar estos datos.

In [ ]:
restaurants_google_fallas = restaurants_google[restaurants_google.index.isin(listado_fallas)]
restaurants_google_fallas.reset_index(inplace=True)
restaurants_google_fallas.info()

In [ ]:
listado_fallas = []
for i in range(restaurants_google_fallas['address'].values.size):
    try:
        lista_address = restaurants_google_fallas['address'].values[i].split(',')[0:4]      #Cortamos el "United States" de los -->
        int(lista_address[-1][-1])                                                          #registros que los tienen.
        if len(lista_address[-1].strip()) == 8:
            localizaciones.append([restaurants_google_fallas['gmap_id'].values[i],          #Id del restaurant
                                   lista_address[-2].strip(),                               #Ciudad
                                   lista_address[-1].strip().split()[0],                    #Estado
                                   lista_address[-1].strip().split()[1]])                   #Codigo postal
        else:
            listado_fallas.append(i)                                    
    except:
        listado_fallas.append(i)
len(listado_fallas)

In [ ]:
#Armamos dataframe con estos datos
localizacion = pd.DataFrame(columns=['gmap_id','Ciudad','Estado','Cod_postal'],data=localizaciones)
localizacion.head()

In [ ]:
#Realizamos una comprobación. Vemos si algún valor falló
fallados = []
for i in range(localizacion.shape[0]):
    try:
        int(localizacion['Cod_postal'].iloc[i])                 #Si esto funciona quiere decir que no entraron caracteres extraños
    except:                                                     #bajo cod_postal
        print(i,': ',localizacion['Cod_postal'].iloc[i])
        fallados.append(localizacion['Cod_postal'].iloc[i])

In [ ]:
#Se redujeron a 1008 casos nomas. Volvemos a observar.
restaurants_google_fallas[restaurants_google_fallas.index.isin(listado_fallas)]

In [ ]:
#Observamos el campo address mas de cerca
list(restaurants_google_fallas[restaurants_google_fallas.index.isin(listado_fallas)]['address'].values)

Otra gran cantidad de registros tiene el código postal adelante de todo, seguido por el estado (nombre completo, no simbolo) y luego por la ciudad. Hacemos un último algoritmo para rescatar estos. Para el resto no se ve ningún patrón.

In [ ]:
#Descartamos las columnas de indices que se nos fueron creando
restaurants_google_fallas.drop(columns=['level_0','index'],inplace=True)
restaurants_google_fallas.info()

In [ ]:
#Creamos un nuevo dataframe
restaurants_google_fallas2 = restaurants_google_fallas[restaurants_google_fallas.index.isin(listado_fallas)]
restaurants_google_fallas2.reset_index(inplace=True)
restaurants_google_fallas2.info()

In [ ]:
#Primero cambiamos los nombres de los estados por sus códigos
state_codes = {'Alabama': 'AL','Alaska': 'AK','Arizona': 'AZ','Arkansas': 'AR','California': 'CA','Colorado': 'CO','Connecticut': 'CT',
    'Delaware': 'DE','Florida': 'FL','Georgia': 'GA','Hawaii': 'HI','Idaho': 'ID','Illinois': 'IL','Indiana': 'IN','Iowa': 'IA',
    'Kansas': 'KS','Kentucky': 'KY','Louisiana': 'LA','Maine': 'ME','Maryland': 'MD','Massachusetts': 'MA','Michigan': 'MI',
    'Minnesota': 'MN','Mississippi': 'MS','Missouri': 'MO','Montana': 'MT','Nebraska': 'NE','Nevada': 'NV','New Hampshire': 'NH',
    'New Jersey': 'NJ','New Mexico': 'NM','New York': 'NY','North Carolina': 'NC','North Dakota': 'ND','Ohio': 'OH','Oklahoma': 'OK',
    'Oregon': 'OR','Pennsylvania': 'PA','Rhode Island': 'RI','South Carolina': 'SC','South Dakota': 'SD','Tennessee': 'TN','Texas': 'TX',
    'Utah': 'UT','Vermont': 'VT','Virginia': 'VA','Washington': 'WA','West Virginia': 'WV','Wisconsin': 'WI','Wyoming': 'WY'}

for state, code in state_codes.items():
    restaurants_google_fallas2['address'] = restaurants_google_fallas2['address'].str.replace(state,code)
restaurants_google_fallas2

In [ ]:
#Quitamos los caracteres extraños
for i in range(restaurants_google_fallas2.shape[0]):
    restaurants_google_fallas2['address'][i] = re.sub(r'[^a-zA-Z0-9,\s]', '', restaurants_google_fallas2['address'][i])
restaurants_google_fallas2.head()

In [ ]:
listado_fallas = []
for i in range(restaurants_google_fallas2['address'].values.size):
    try:
        int(restaurants_google_fallas2.iloc[i]['address'][0])         #Si esto sucede entonces el primer digito es un numero
        localizaciones.append([restaurants_google_fallas2['gmap_id'].values[i],                                     #Id del restaurant
                            restaurants_google_fallas2['address'].values[i].split(',')[1].strip(),                 #Ciudad
                            restaurants_google_fallas2['address'].values[i].split(',')[0].strip().split()[1],      #Estado
                            restaurants_google_fallas2['address'].values[i].split(',')[0].strip().split()[0]])     #Código postal
    except:
        listado_fallas.append(i)                                        #Anoto cuantas direcciones no me dan los datos necesarios
len(listado_fallas)

In [ ]:
#Armamos dataframe con estos datos
localizacion = pd.DataFrame(columns=['gmap_id','Ciudad','Estado','Cod_postal'],data=localizaciones)
localizacion.head()

In [ ]:
#Realizamos una comprobación. Vemos si algún valor falló
fallados = []
for i in range(localizacion.shape[0]):
    try:
        int(localizacion['Cod_postal'].iloc[i])                 #Si esto funciona quiere decir que no entraron caracteres extraños
    except:                                                     #bajo cod_postal
        print(i,': ',localizacion['Cod_postal'].iloc[i])
        fallados.append(i)

In [ ]:
#Juntamos con el de google
restaurants_google = pd.merge(google,localizacion,how='left',on='gmap_id')
restaurants_google.head()

In [ ]:
restaurants_google.info()

Terminada esta parte vamos a armar el dataframe que deseamos. Descartamos las columnas que no sirven, cambiamos nombre, reordenamos, etc.

In [ ]:
#Descartamos columnas
restaurants_google.drop(columns=['description','avg_rating','num_of_reviews','price',
                                 'hours','state','relative_results','url','address'], inplace=True)
restaurants_google.head()

In [ ]:
#Cambiamos nombre
restaurants_google.rename(columns={'name':'Nombre','gmap_id':'Id_Restaurant','latitude':'Latitud','longitude':'Longitud',
                                   'category':'Tipo','MISC':'Atributos'},inplace=True)
restaurants_google.head()

In [ ]:
#Reordenamos
restaurants_google = restaurants_google[['Id_Restaurant','Nombre','Ciudad','Estado','Cod_postal', 
                                         'Latitud', 'Longitud', 'Tipo', 'Atributos']]
restaurants_google.head()

In [ ]:
#Observamos el resultado
restaurants_google.info()

Hay 1830 valores nulos de los campos Ciudad, Estado y Cod_Postal. Es menos del 1% del dataset y el trabajo para completar esto es demasiado. Vamos a descartar estos registros tanto de aquí como de las reviews.

In [ ]:
#Descartamos
restaurants_google.dropna(subset='Cod_postal',inplace=True)
restaurants_google.reset_index(inplace=True)
restaurants_google.drop(columns='index',inplace=True)
restaurants_google.info()

In [ ]:
#Vamos a observar si hay negocios que se encuentren duplicados.
#Es decir que tengan distinto Id (ya se descartó antes los de igual Id) pero coincidan nombre y ubicación.
duplicados_google = restaurants_google[restaurants_google.duplicated(subset=['Nombre','Cod_postal'],keep=False)].sort_values(by='Nombre')
duplicados_google.head()

In [ ]:
#Vemos que hay y tienen la misma ubicación. Vemos cuantos son.
duplicados_google.reset_index(inplace=True)
duplicados_google.info()

In [ ]:
#Vemos cuantos negocios son
negocios_duplicados_google = duplicados_google.drop_duplicates(subset=['Nombre','Cod_postal'])
negocios_duplicados_google.info()

Vamos a quedarnos con uno de los duplicados y descartar los otros. Sin embargo, vamos a cambiar primero el id de las reseñas así no perdemos información.

In [ ]:
#Tomamos los ids a conservar y los que se van a cambiar
lista_duplicados = []
for n in negocios_duplicados_google['Id_Restaurant'].values:
    aux = []
    nombre = negocios_duplicados_google[negocios_duplicados_google['Id_Restaurant']==n]['Nombre'].values[0]
    cod_postal = negocios_duplicados_google[negocios_duplicados_google['Id_Restaurant']==n]['Cod_postal'].values[0]
    for d in range(duplicados_google.shape[0]):
        if (duplicados_google['Nombre'][d] == nombre) and (duplicados_google['Cod_postal'][d] == cod_postal) and (duplicados_google['Id_Restaurant'][d] != n):
            aux.append(duplicados_google['Id_Restaurant'][d])
    lista_duplicados.append([n,aux])

In [ ]:
#Armammos dataframe
duplicados_google_df = pd.DataFrame(data=lista_duplicados,columns=['Conservar','Cambiar'])
duplicados_google_df.head()

In [ ]:
#Descartamos los duplicados
restaurants_google.drop_duplicates(subset=['Nombre','Cod_postal'],inplace=True)
restaurants_google.reset_index(inplace=True)
restaurants_google.drop(columns='index',inplace=True)
restaurants_google.info()

In [ ]:
#Exportamos
restaurants_google.to_pickle('restaurants_google_final.pkl')